In [1]:
import ccapi

config         = ccapi.Configuration()
config.url     = "http://localhost:5000"
config.verbose = True

### Building a Boolean-Based Model

This simple example demonstrates how to create a boolean-based model, its components, regulators, conditions and sub-conditions into [Cell Collective](https://cellcollective.org).

We'll attempt to reconstruct the [Cortical Area Development](https://research.cellcollective.org/?dashboard=true#2035:1/cortical-area-development/1) authored by CE Giacomantonio.

![](../data/images/cortical-area-development.png)

Begin by importing the ccapi module into your workspace.

In [2]:
import ccapi

Now, let’s try creating a client object in order to interact with services provided by [Cell Collective](https://cellcollective.org).

In [3]:
client = ccapi.Client()

2020-01-27 17:06:07,531 | INFO | Dispatching a GET request to URL: http://localhost:5000/api/ping with Arguments - {}


Authenticate your client using a ***password flow type authentication*** scheme.

**NOTE**: *Before you can authenticate using ccapi, you must first register an application of the appropriate type on [Cell Collective](https://cellcollective.org). If you do not require a user context, it is read only.*

In [4]:
def authenticate():
    client.auth(email = "test@cellcollective.org", password = "test")

try:
    authenticate()
except ccapi.exception.AuthenticationError:
    client.sign_up(
        email       = "test@cellcollective.org",
        password    = "test",
        first_name  = "Test",
        last_name   = "Test",
        institution = "Test"
    )
    
    authenticate()

2020-01-27 17:06:07,862 | INFO | Dispatching a POST request to URL: http://localhost:5000/_api/login with Arguments - {'data': {'username': 'test@cellcollective.org', 'password': 'test'}}


#### Creating a Base Model

Create a Base Model using ccapi and instantize it with an authenticated client.

In [5]:
model = ccapi.Model("Cortical Area Development", client = client)
model.save()

2020-01-27 17:06:08,447 | INFO | Dispatching a GET request to URL: http://localhost:5000/_api/user/getProfile with Arguments - {}
2020-01-27 17:06:08,492 | INFO | Dispatching a POST request to URL: http://localhost:5000/_api/model/save with Arguments - {'json': {'-43792/-27709': {'name': 'Cortical Area Development', 'type': 'research', 'userId': 10887, 'modelVersionMap': {-27709: {'name': ''}}, 'speciesMap': {}, 'regulatorMap': {}, 'conditionMap': {}, 'conditionSpeciesMap': {}, 'subConditionMap': {}, 'subConditionSpeciesMap': {}}}}
2020-01-27 17:06:08,539 | INFO | Searching for templates within directories: ['/home/achilles/dev/ccapi/src/ccapi/data/templates']


ID,90943
Name,Cortical Area Development
Memory Address,0x07fc6c1732b70
Number of Versions,1
Versions,[<BooleanModel 90943 version 1 at 0x07fc6c1732d68>]


A `ccapi.Model` consists of various `ccapi.ModelVersion` objects that help you build various versions to a model. By default, a `ccapi.Model` provides you a default model version of a boolean type. 

In [6]:
# get the default model version
boolean      = model.versions[0]
boolean.name = "Version 1"
boolean

2020-01-27 17:06:08,916 | INFO | Searching for templates within directories: ['/home/achilles/dev/ccapi/src/ccapi/data/templates']


ID,90943
Version,1
Name,Version 1
Memory Address,0x07fc6c1732d68
Number of Components,0
Components,


#### Adding Components to a Boolean-Based Model

First, we need to create a list of component objects for this model.

In [7]:
# create components
COUP_TFI = ccapi.InternalComponent("COUP-TFI")
EMX2     = ccapi.InternalComponent("EMX2")
FGF8     = ccapi.InternalComponent("FGF8")
PAX6     = ccapi.InternalComponent("PAX6")
Sp8      = ccapi.InternalComponent("Sp8")

Now let us add a list of components to our Boolean Model.

In [8]:
# add components to model
boolean.add_components(COUP_TFI, EMX2, FGF8, PAX6, Sp8)

#### Saving a Model

Ensure you save your model in order to commit your work.

In [9]:
model.save()

2020-01-27 17:06:11,240 | INFO | Dispatching a GET request to URL: http://localhost:5000/_api/user/getProfile with Arguments - {}
2020-01-27 17:06:11,270 | INFO | Dispatching a POST request to URL: http://localhost:5000/_api/model/save with Arguments - {'json': {'90943/1': {'name': 'Cortical Area Development', 'type': 'research', 'userId': 10887, 'modelVersionMap': {1: {'name': 'Version 1'}}, 'speciesMap': {-56220: {'name': 'COUP-TFI', 'external': False}, -2178: {'name': 'EMX2', 'external': False}, -39396: {'name': 'FGF8', 'external': False}, -56252: {'name': 'PAX6', 'external': False}, -2019: {'name': 'Sp8', 'external': False}}, 'regulatorMap': {}, 'conditionMap': {}, 'conditionSpeciesMap': {}, 'subConditionMap': {}, 'subConditionSpeciesMap': {}}}}
2020-01-27 17:06:11,348 | INFO | Searching for templates within directories: ['/home/achilles/dev/ccapi/src/ccapi/data/templates']


ID,90943
Name,Cortical Area Development
Memory Address,0x07fc6c1732b70
Number of Versions,1
Versions,[<BooleanModel 90943 version 1 at 0x07fc6c1732d68 name='Version 1'>]


#### Adding Regulators, Conditions and Sub-Conditions

Let's add a list of regulators and conditions to our components. A list of regulators and conditions as well as sub-conditions can all be added at once to a component.

In [10]:
# add regulators to components
COUP_TFI.add_regulators(
    ccapi.NegativeRegulator(Sp8),
    ccapi.NegativeRegulator(FGF8)
)
EMX2.add_regulators(
    ccapi.PositiveRegulator(COUP_TFI),
    ccapi.NegativeRegulator(FGF8),
    ccapi.NegativeRegulator(PAX6),
    ccapi.NegativeRegulator(Sp8)
)
Sp8.add_regulators(
    ccapi.PositiveRegulator(FGF8),
    ccapi.NegativeRegulator(EMX2)
)
FGF8.add_regulators(
    # add conditions to regulators
    ccapi.PositiveRegulator(FGF8, conditions = [
        ccapi.Condition(components = Sp8)
    ])
)
PAX6.add_regulators(
    ccapi.PositiveRegulator(Sp8),
    ccapi.NegativeRegulator(COUP_TFI)
)

In [11]:
model.save()

2020-01-27 17:06:12,617 | INFO | Dispatching a GET request to URL: http://localhost:5000/_api/user/getProfile with Arguments - {}
2020-01-27 17:06:12,648 | INFO | Dispatching a POST request to URL: http://localhost:5000/_api/model/save with Arguments - {'json': {'90943/1': {'name': 'Cortical Area Development', 'type': 'research', 'userId': 10887, 'modelVersionMap': {1: {'name': 'Version 1'}}, 'speciesMap': {1859453: {'name': 'COUP-TFI', 'external': False}, 1859450: {'name': 'EMX2', 'external': False}, 1859452: {'name': 'FGF8', 'external': False}, 1859454: {'name': 'PAX6', 'external': False}, 1859451: {'name': 'Sp8', 'external': False}}, 'regulatorMap': {-18117: {'regulationType': 'NEGATIVE', 'regulatorSpeciesId': 1859451, 'speciesId': 1859453}, -32510: {'regulationType': 'NEGATIVE', 'regulatorSpeciesId': 1859452, 'speciesId': 1859453}, -28063: {'regulationType': 'POSITIVE', 'regulatorSpeciesId': 1859453, 'speciesId': 1859450}, -55985: {'regulationType': 'NEGATIVE', 'regulatorSpeciesId'

ID,90943
Name,Cortical Area Development
Memory Address,0x07fc6c1732b70
Number of Versions,1
Versions,[<BooleanModel 90943 version 1 at 0x07fc6c1732d68 name='Version 1'>]


We've now got things within our Boolean Model.

In [12]:
boolean.components

ID,Name,Number of Positive Regulators,Number of Negative Regulators
1859453,COUP-TFI,0,2
1859450,EMX2,1,3
1859452,FGF8,1,0
1859454,PAX6,1,1
1859451,Sp8,1,1


In [13]:
FGF8.positive_regulators

ID,Name,Number of Conditions
2552170,,1


#### Model Summary

You can view detailed summary of your model using the `summary` function provided.

In [14]:
boolean.summary()

Internal Components (+, -) External Components
-------------------------- -------------------
COUP-TFI (0,2)             
EMX2 (1,3)                 
FGF8 (1,0)                 
PAX6 (1,1)                 
Sp8 (1,1)                  


...or view detailed information within your jupyter notebook.

In [15]:
boolean

2020-01-27 17:06:16,393 | INFO | Searching for templates within directories: ['/home/achilles/dev/ccapi/src/ccapi/data/templates']


ID,90943
Version,1
Name,Version 1
Memory Address,0x07fc6c1732d68
Number of Components,5
Components,"COUP-TFI, EMX2, FGF8, PAX6, Sp8"


#### Model Rendering

You can also attempt to visualize a Boolean Model using the `draw` function provided.

In [ ]:
# boolean.draw()